In [1]:
import pandas as pd
df=pd.read_csv("abcd - data.csv")

In [2]:
df.head()

,Email_Subject,Job_Application
0,Thank you for applying to Dealozo for internship,Yes
1,A third-party OAuth application has been added...,No
2,Auto response: AWS Educate Application Not App...,No
3,Thank you for applying to Stirring Minds,Yes
4,Earn your MCS from ASU - Early application dea...,No


In [3]:
df['label'] = df['Job_Application'].apply(lambda x: 0 if x=='No' else 1)

In [4]:
df.head()

,Email_Subject,Job_Application,label
0,Thank you for applying to Dealozo for internship,Yes,1
1,A third-party OAuth application has been added...,No,0
2,Auto response: AWS Educate Application Not App...,No,0
3,Thank you for applying to Stirring Minds,Yes,1
4,Earn your MCS from ASU - Early application dea...,No,0


In [7]:
!pip install spacy

!python -m spacy download en

     |████████████████████████████████| 9.9 MB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 30.1 MB/s eta 0:00:01    |▊                               | 51 kB 23.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.7 MB 41.8 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 41.3 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 21.8 MB/s eta 0:00:01
     |████████████████████████████████| 185 kB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 12.0 MB 13.2 MB/s eta 0:00:01   |███████████████▍                | 5.8 MB 358 kB/s eta 0:00:18
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=54dd6f76ddeea3db1f010f36a1979d370bbd966dfab44b666ef43ac4e5b8208b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3lpiauhy/wheels/10/6f/a6/ddd8204ceecdedddea923f8514e13afb0c1f0f556d2c9c3da0
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load th

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [33]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens
#spacy_tokenizer(sentence)

NameError: name 'sentence' is not defined

In [11]:
from spacy.lang.en import English

In [12]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [35]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [36]:
from sklearn.model_selection import train_test_split

X = df['Email_Subject'] # the features we want to analyze
ylabels = df['label'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [37]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f6e73b96cc0>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f6e960b1840>)),
                ('classifier', LogisticRegression())])

In [38]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9375
Logistic Regression Precision: 1.0
Logistic Regression Recall: 0.8888888888888888


array([1])

In [49]:
[X_test, y_test]

[22     Your application status update from September 14
 3             Thank you for applying to Stirring Minds 
 5                   Thank you for applying to Strings73
 31                Thank You for applying at TCS Careers
 17                Thank you for applying to AIMonk Labs
 46    3 internships mandatory to get your degree - H...
 14                   Thanks for your application, Ajith
 11    Thank you for applying and completing the appl...
 12                    Thank you for applying at Siemens
 36    Thank you for applying in the role of Data Sci...
 28    Vaseeswaran S.P upvoted your answer to: How do...
 50    Ajith, represent IIT Delhi, IIT Roorkee, IIT M...
 48    National Doctors' Day: A heartfelt thanks to a...
 40                         There's still time to apply!
 21                Thank You for applying to Whatscutpro
 38                      Skills you can build in 6 weeks
 Name: Email_Subject, dtype: object,
 22    0
 3     1
 5     1
 31    1
 17    1
 46   

In [50]:
pipe.predict([""])

array([0])